In [53]:
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd
import numpy as np
import html5lib
import requests
from openpyxl import Workbook
from datetime import datetime
import webbrowser
import os


def parse_and_trim(content, content_type):
    if content_type == "HTML":
        soup = BeautifulSoup(content, 'html.parser')
    else:
        soup = BeautifulSoup(content, 'html.parser')
    for tag in soup.recursiveChildGenerator():
        try:
            tag.attrs = None
        except AttributeError:
            pass
    for linebreak in soup.find_all('br'):
        linebreak.extract()
    return soup

In [58]:
headers = {
    'User-Agent': 'GOLUB CAPITAL BDC, Inc.'
}
filing_links = pd.read_excel('../MFIC__sec_filing_links.xlsx')
date_columns = ['Filing date', 'Reporting date']
for col in date_columns:
    filing_links[col] = pd.to_datetime(filing_links[col], format='%Y-%m-%d')
for col in date_columns:
    filing_links[col] = filing_links[col].dt.strftime("%B %d, %Y")
filing_links.head(5)

,Form type,Form description,Filing date,Reporting date,url
0,10-Q,Quarterly report [Sections 13 or 15(d)],"November 07, 2023","September 30, 2023",https://www.sec.gov/Archives/edgar/data/127875...
1,10-Q,Quarterly report [Sections 13 or 15(d)],"August 02, 2023","June 30, 2023",https://www.sec.gov/Archives/edgar/data/127875...
2,10-Q,Quarterly report [Sections 13 or 15(d)],"May 02, 2023","March 31, 2023",https://www.sec.gov/Archives/edgar/data/127875...
3,10-KT,Transition reports [Rule 13a-10 or 15d-10],"February 21, 2023","December 31, 2022",https://www.sec.gov/Archives/edgar/data/127875...
4,10-Q,Quarterly report [Sections 13 or 15(d)],"November 03, 2022","September 30, 2022",https://www.sec.gov/Archives/edgar/data/127875...


In [59]:
def extract_tables(soup_content, qtr_date):
    master_table = None
    print(qtr_date)
    # if qtr_date == 'December 31, 2015' or qtr_date == 'June 30, 2016':
    #     consolidated_schedule_regex = re.compile(
    #         r'(?i)^\s*.*\s*CONSOLIDATED\s+SCHEDULE(S|)\s+OF\s+INVESTMENTS\s*(\(.*\)|)\s*-.*\s*\(.*\)$')
    # else:
    consolidated_schedule_regex = re.compile(
        r'(?i)^\s*.*\s*CONSOLIDATED\s+SCHEDULE(S|)\s+OF\s+INVESTMENTS\s*.*\s*$')
    date_regex_pattern = r'([A-Za-z]+\s+\d{1,2},\s+\d{4})'
    # date_regex_pattern2 = r'\bAs\s+of\s+([A-Za-z]+\s+\d{1,2},\s+\d{4})\b'
    for tag in soup_content.find_all(text=re.compile(consolidated_schedule_regex)):
        date_str = re.search(date_regex_pattern, tag.find_next().text)
        if date_str is None:
            date_str = re.search(date_regex_pattern, tag.next.text)
        if date_str is None:
            date_str = re.search(date_regex_pattern,
                                 tag.find_next().next.next.next.text)
        if date_str is not None:
            date_str = str(date_str.group(1))
            date_str = unicodedata.normalize('NFKD', date_str)
            qtr_date_cleaned = qtr_date.replace(',', '').replace(
                ' ', '').replace('\n', '').lower()
            date_str_cleaned = date_str.replace(',', '').replace(
                ' ', '').replace('\n', '').lower()
            print(date_str, qtr_date_cleaned, date_str_cleaned)

            if qtr_date_cleaned == date_str_cleaned:
                html_table = tag.find_next('table')
                new_table = pd.read_html(
                    html_table.prettify(), keep_default_na=False, skiprows=0, flavor='bs4')[0]
                new_table = new_table.applymap(lambda x: unicodedata.normalize(
                    'NFKD', x.strip().strip(u'\u200b').replace('-', '0').replace('%', '')) if type(x) == str else x)

                new_table = new_table.replace(
                    r'^\s*$', np.nan, regex=True).replace(r'^\s*\$\s*$', np.nan, regex=True)
                new_table = new_table.dropna(how='all', axis=0)

                if master_table is None:
                    master_table = new_table
                else:
                    master_table = pd.concat(
                        [master_table, new_table], ignore_index=True)
            # print(master_table)

    master_table = master_table.applymap(
        lambda x: x.strip().strip(u'\u200b') if type(x) == str else x)
    master_table = master_table.replace(r'^\s*$', np.nan, regex=True).replace(
        r'^\s*\$\s*$', np.nan, regex=True).replace(r'^\s*\)\s*$', np.nan, regex=True)
    print(master_table.shape)
    return master_table

In [60]:
master = extract_tables(
    filing_links['url'][0], filing_links['Reporting date'][0])

September 30, 2023


AttributeError: 'str' object has no attribute 'find_all'

'https://www.sec.gov/Archives/edgar/data/1278752/000095017023060325/mfic-20230930.htm'